In [1]:
import os
import random
import shutil
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import albumentations as A
from tqdm.auto import tqdm

ModuleNotFoundError: No module named 'albumentations'

In [6]:
BASE_DIR = 'learning'
FONT_DIR = os.path.join(BASE_DIR, 'fonts')
FONT_DIR_VALID = os.path.join(BASE_DIR, 'fonts_valid')
DATA_DIR = os.path.join(BASE_DIR, 'data')
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')

# Параметри генерації
IMAGE_SIZE = (64, 64)
CLASSES = list(range(10))
NEGATIVE_CLASS_ID = 10
FONT_SIZES = [38, 42, 46, 50, 54] # 5 варіантів розмірів

RANDOM_AUGMENTATIONS_PER_IMAGE = 50
NEGATIVE_TEST_SAMPLES = 5000 # Зробимо ~баланс з позитивними

# Очищення старої директорії TEST
if os.path.exists(TEST_DIR):
    print(f"Очищую стару директорію: {TEST_DIR}...")
    shutil.rmtree(TEST_DIR)

# Створення нової структури (включаючи клас "10")
for c in CLASSES + [NEGATIVE_CLASS_ID]:
    os.makedirs(os.path.join(TEST_DIR, str(c)), exist_ok=True)

print(f"Структуру директорій TEST створено в {TEST_DIR}")

Очищую стару директорію: learning\data\test...
Структуру директорій TEST створено в learning\data\test


In [4]:
print("--- Етап 2: Налаштування генераторів ---")

# --- Базовий генератор (з фоном і тремтінням) ---
def create_base_image(digit_char, font_path, font_size, image_size=(64, 64)):
    bg_noise = np.random.randint(0, 40, image_size, dtype=np.uint8)
    image = Image.fromarray(bg_noise, 'L')
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        font = ImageFont.load_default()
    text_width, text_height = draw.textbbox((0,0), digit_char, font=font)[2:4]
    x = (image_size[0] - text_width) / 2 + random.randint(-4, 4)
    y = (image_size[1] - text_height) / 2 + random.randint(-4, 4)
    draw.text((x, y), digit_char, font=font, fill=random.randint(220, 255))
    return np.array(image)

# --- Генератор "сміття" ---
def create_garbage_image(image_size=(64, 64), font_files_list=None):
    garbage_type = random.choice(['noise', 'lines', 'letters', 'extreme_aug'])
    image_np = np.random.randint(0, 50, image_size, dtype=np.uint8)
    image = Image.fromarray(image_np, 'L')
    draw = ImageDraw.Draw(image)

    if garbage_type == 'lines':
        for _ in range(random.randint(1, 3)):
            x1, y1 = random.randint(0, 64), random.randint(0, 64)
            x2, y2 = random.randint(0, 64), random.randint(0, 64)
            draw.line((x1, y1, x2, y2), fill=random.randint(100, 255), width=random.randint(1, 3))
        image_np = np.array(image)

    elif garbage_type == 'letters' and font_files_list:
        try:
            font_path = random.choice(font_files_list)
            font_size = random.choice(FONT_SIZES)
            font = ImageFont.truetype(font_path, font_size)
            char = random.choice("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")
            draw.text((5, 5), char, font=font, fill=random.randint(150, 255))
        except Exception:
            pass
        image_np = np.array(image)

    elif garbage_type == 'extreme_aug':
        aug = random.choice([A.GridDistortion(p=1.0), A.Blur(blur_limit=15, p=1.0)])
        image_np = aug(image=image_np)['image']

    return image_np


random_aug_pipeline = A.Compose([
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.OneOf([
        A.MotionBlur(blur_limit=(3, 10), p=1.0),
        A.GridDistortion(num_steps=5, distort_limit=0.1, p=1.0),
    ], p=0.5),
    A.Perspective(scale=(0.05, 0.1), pad_mode=0, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.Blur(blur_limit=(3, 7), p=0.3),
], p=1.0) # Застосувати комбінацію до кожного зображення

print("Генератори та аугментації налаштовано.")

--- Етап 2: Налаштування генераторів ---
Генератори та аугментації налаштовано.


C:\Users\perce\AppData\Local\Temp\ipykernel_4616\1994726421.py:51: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
C:\Users\perce\miniconda3\Lib\site-packages\albumentations\augmentations\blur\functional.py:232: UserWarning: blur_limit: Non-zero kernel sizes must be odd. Range (3, 10) automatically adjusted to (3, 11).
  result = _ensure_odd_values(result, info.field_name)
C:\Users\perce\AppData\Local\Temp\ipykernel_4616\1994726421.py:56: UserWarning: Argument(s) 'pad_mode' are not valid for transform Perspective
  A.Perspective(scale=(0.05, 0.1), pad_mode=0, p=0.5),
C:\Users\perce\miniconda3\Lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [7]:
print("--- Етап 3: Генерація TEST сету ---")

# Завантажуємо список шрифтів з VALID папки
try:
    valid_font_files = [os.path.join(FONT_DIR_VALID, f) for f in os.listdir(FONT_DIR_VALID) if f.endswith('.ttf')]
    if not valid_font_files:
        raise FileNotFoundError
except FileNotFoundError:
    print(f"ПОМИЛКА: Директорія {FONT_DIR_VALID} порожня або не знайдена. Зупинка.")
    # Тут можна зупинити блокнот або вийти
    raise

# 1. Створюємо "базовий" набір завдань для тесту
base_test_tasks = []
for digit in CLASSES:
    for font_path in valid_font_files:
        for font_size in FONT_SIZES:
            base_test_tasks.append({
                "digit": digit,
                "font_path": font_path,
                "font_size": font_size,
                "id": f"{digit}_font_{os.path.basename(font_path).split('.')[0]}_size_{font_size}"
            })

total_base_images = len(base_test_tasks)
print(f"Знайдено {len(valid_font_files)} валідаційних шрифтів.")
print(f"Всього буде {total_base_images} базових зображень (10 цифр * {len(valid_font_files)} шрифтів * {len(FONT_SIZES)} розмірів).")

--- Етап 3: Генерація TEST сету ---
Знайдено 2 валідаційних шрифтів.
Всього буде 100 базових зображень (10 цифр * 2 шрифтів * 5 розмірів).


In [8]:
print(f"Генерую 1 чистий + {RANDOM_AUGMENTATIONS_PER_IMAGE} аугментованих зразків для кожного.")

# 2. Генеруємо TEST сет (позитивні класи 0-9)
for task in tqdm(base_test_tasks, desc="Генерація TEST (0-9)"):
    digit = task['digit']
    base_id = task['id']

    # Генеруємо базове зображення
    base_img_np = create_base_image(str(digit), task['font_path'], task['font_size'])

    # a) Зберігаємо "чисту" версію
    save_path_clean = os.path.join(TEST_DIR, str(digit), f"{base_id}_clean.png")
    Image.fromarray(base_img_np).save(save_path_clean)

    # b) Генеруємо N випадкових аугментованих версій
    for i in range(RANDOM_AUGMENTATIONS_PER_IMAGE):
        augmented_img_np = random_aug_pipeline(image=base_img_np)['image']

        save_path_aug = os.path.join(TEST_DIR, str(digit), f"{base_id}_aug_{i:03d}.png")
        Image.fromarray(augmented_img_np).save(save_path_aug)

total_positive = total_base_images * (1 + RANDOM_AUGMENTATIONS_PER_IMAGE)
print(f"Генерацію позитивного TEST сету завершено. Всього: {total_positive} зображень.")

Генерую 1 чистий + 50 аугментованих зразків для кожного.


Генерація TEST (0-9): 100%|██████████| 100/100 [00:11<00:00,  8.77it/s]

Генерацію позитивного TEST сету завершено. Всього: 5100 зображень.


In [9]:
print(f"\n--- Етап 4: Генерація TEST сету (Негативний клас {NEGATIVE_CLASS_ID}) ---")
for i in tqdm(range(NEGATIVE_TEST_SAMPLES), desc="Генерація 'сміття' (Test)"):
    # Передаємо валідаційні шрифти, щоб літери були з "небачених" шрифтів
    garbage_img_np = create_garbage_image(IMAGE_SIZE, valid_font_files)
    save_path = os.path.join(TEST_DIR, str(NEGATIVE_CLASS_ID), f"garbage_{i:05d}.png")
    Image.fromarray(garbage_img_np).save(save_path)

print(f"Згенеровано {NEGATIVE_TEST_SAMPLES} 'сміттєвих' зображень для TEST.")
print(f"\n--- ✅ УСЮ ГЕНЕРАЦІЮ TEST СЕТУ ЗАВЕРШЕНО! ---")
print(f"Загалом у data/test: {total_positive + NEGATIVE_TEST_SAMPLES} файлів.")


--- Етап 4: Генерація TEST сету (Негативний клас 10) ---


Генерація 'сміття' (Test): 100%|██████████| 5000/5000 [00:07<00:00, 670.45it/s]

Згенеровано 5000 'сміттєвих' зображень для TEST.

--- ✅ УСЮ ГЕНЕРАЦІЮ TEST СЕТУ ЗАВЕРШЕНО! ---
Загалом у data/test: 10100 файлів.


In [15]:
print(f"\n--- Етап 5: Генерація TRAIN сету (Негативний клас {NEGATIVE_CLASS_ID}) ---")
for i in tqdm(range(NEGATIVE_TRAIN_SAMPLES), desc="Генерація 'сміття' (Train)"):
    garbage_img_np = create_garbage_image(IMAGE_SIZE)
    save_path = os.path.join(TRAIN_DIR, str(NEGATIVE_CLASS_ID), f"garbage_{i:05d}.png")
    Image.fromarray(garbage_img_np).save(save_path)

print(f"Згенеровано {NEGATIVE_TRAIN_SAMPLES} 'сміттєвих' зображень для TRAIN.")


--- Етап 5: Генерація TRAIN сету (Негативний клас 10) ---


Генерація 'сміття' (Train):   0%|          | 0/25000 [00:00<?, ?it/s]C:\Users\perce\miniconda3\Lib\site-packages\albumentations\augmentations\blur\functional.py:232: UserWarning: blur_limit: Non-zero kernel sizes must be odd. Range (3, 20) automatically adjusted to (3, 21).
  result = _ensure_odd_values(result, info.field_name)
Генерація 'сміття' (Train): 100%|██████████| 25000/25000 [01:20<00:00, 308.65it/s]

Згенеровано 25000 'сміттєвих' зображень для TRAIN.


In [16]:
print(f"Генерую {len(test_tasks)} 'чистих' тестових зображень...")

for task in tqdm(test_tasks, desc="Генерація TEST (0-9)"):
    digit = task['digit']
    base_id = task['id']

    # Генеруємо ТІЛЬКИ базове зображення (з шумом фону і тремтінням)
    base_img_np = create_base_image(str(digit), task['font_path'], task['font_size'])

    # Зберігаємо
    save_path = os.path.join(TEST_DIR, str(digit), f"{base_id}_clean.png")
    Image.fromarray(base_img_np).save(save_path)

print("Генерацію позитивного TEST сету завершено.")

# 6. Генеруємо TEST сет (негативний клас 10)
print(f"\n--- Етап 7: Генерація TEST сету (Негативний клас {NEGATIVE_CLASS_ID}) ---")
for i in tqdm(range(NEGATIVE_TEST_SAMPLES), desc="Генерація 'сміття' (Test)"):
    garbage_img_np = create_garbage_image(IMAGE_SIZE)
    save_path = os.path.join(TEST_DIR, str(NEGATIVE_CLASS_ID), f"garbage_{i:05d}.png")
    Image.fromarray(garbage_img_np).save(save_path)

print(f"Згенеровано {NEGATIVE_TEST_SAMPLES} 'сміттєвих' зображень для TEST.")
print("\n--- ✅ УСЮ ГЕНЕРАЦІЮ ЗАВЕРШЕНО! ---")

Генерую 50 'чистих' тестових зображень...


Генерація TEST (0-9): 100%|██████████| 50/50 [00:02<00:00, 19.19it/s]


Генерацію позитивного TEST сету завершено.

--- Етап 7: Генерація TEST сету (Негативний клас 10) ---


Генерація 'сміття' (Test): 100%|██████████| 1000/1000 [00:11<00:00, 90.64it/s]

Згенеровано 1000 'сміттєвих' зображень для TEST.

--- ✅ УСЮ ГЕНЕРАЦІЮ ЗАВЕРШЕНО! ---
